## this notebook was used for testing google cloud function code for single_shot_sbir
## some modificaitons were made to run in functions-framework
## this notebook was replaced with one on colab so caleb could modify it

# imports

In [1]:
import requests
import json
import yaml
import sys
sys.path.append('../grantedai')
from chat_complete import chat_complete_openai

# grab api key from cloud

In [4]:
from google.cloud import secretmanager

project_id = 'python-backend-v1'
client = secretmanager.SecretManagerServiceClient()
name = f"projects/{project_id}/secrets/OPENAI_API_KEY/versions/2"
response = client.access_secret_version(request={"name": name})
my_secret_value = response.payload.data.decode("UTF-8")

RetryError: Deadline of 60.0s exceeded while calling target function, last exception: 503 Getting metadata from plugin failed with error: Reauthentication is needed. Please run `gcloud auth application-default login` to reauthenticate.

In [6]:
with open('../grantedai/config/NSF SBIR.yaml','r', encoding='utf-8') as file:
    data = yaml.safe_load(file)

data['NSF - SBIR - 1. Draft Technical Objectives']['include_header']
data.keys()


dict_keys(['document_name', 'draft_order', 'output_order', 'NSF - SBIR - 1. Draft Technical Objectives', 'NSF - SBIR - 2. Project Pitch', 'NSF - SBIR - 3. Expanded Technical Objectives', 'NSF - SBIR - 4. Project Summary', 'NSF - SBIR - 5.1 Project Description: Elevator Pitch', 'NSF - SBIR - 5.2 Project Description: Commercial Opportunity', 'NSF - SBIR - 5.3 Project Description: Technical Solution', 'NSF - SBIR - 5.4 Project Description: Company and Team', 'NSF - SBIR - 5.5 Project Description: Technical Discussion and R&D Plan', 'NSF - SBIR - 5.6 Project Description: Broader Impacts'])

In [4]:
data['NSF - SBIR - 1. Draft Technical Objectives']['qs'].items()

dict_items([('q1input', 'page_q1'), ('q2input', 'page_q3')])

## define what an sbir section is: uid to find the prompt, number of user inputs, what to do with the output, and list of any user inputs that are cut/paste from previous runs

In [3]:
class SbirSection:
    def __init__(self, uid, num_q, result, replace_q = None):
        self.uid = uid
        self.num_q = num_q
        self.result = result # True if the output goes in final sbir, False if its just for cut and paste
        self.replace_q = replace_q  # which qinput is a cut and paste

# this is the structure of the individual pages in the NIH - SBIR section
# uid, number of inputs, what to call the output, dictionary of cut/paste
specific_aims = SbirSection('1686931095568x127537042873881800', 2, "Specific Aims")
concept_overview = SbirSection('1686931095582x126136176460591060', 5, "Output", {4 : "Specific Aims"})
project_narrative = SbirSection('1686931095420x670283125425270500', 1, "Output")
project_summary = SbirSection('1682951835439x155695279795943460', 1, "Output", {1: "Specific Aims"})
personal_statement = SbirSection('1682951835365x179818839606519230', 5, "Output", {5: "Specific Aims"})
contributions_to_science = SbirSection('1682951835291x143168345947523040', 4, "Output")


# this is the collection of nih-sbir pages to be run in order
nih_sbir = [specific_aims, concept_overview, project_narrative, project_summary, personal_statement, contributions_to_science]
    

## these are all the user inputs from nih-sbir pages that are not cut/paste from previous outputs.  15 in total.

In [4]:
# these are the number of user input fields that would be required on an nih-sbir page
qs = []

# Specific aims
# q1 and q2
qs.append("Leverage the unique capabilities of Aldatu’s qPCR-enabling PANDAA technology to mitigate lineage-associated genomic \
variability and develop a rapid, pan-lineage molecular assay for LASV detection")
qs.append("Design PANDAA-LASV primers and probes and optimize reactions.  \
      Refine PANDAA-LASV reagents on divergent genotypes.  \
      Validate PANDAA-LASV diagnostic prototype both analytically and clinically")

# Concept overview
#q3
qs.append("The prevalence of Alzheimer's disease (AD) is rapidly increasing, affecting millions of people \
worldwide. Current diagnostic methods for AD are invasive, expensive, and often inaccurate, leading to \
delayed or missed diagnoses.")

#q4
qs.append("Early and accurate diagnosis of AD is crucial for timely intervention and treatment, \
which can significantly improve patients' quality of life and reduce healthcare costs. Developing a \
non-invasive, cost-effective, and accurate diagnostic tool for AD would greatly benefit patients, \
caregivers, and healthcare providers.")

#skip
qs.append("Our team has developed a novel artificial intelligence (AI)-based system that analyzes speech patterns and \
linguistic markers to detect early signs of AD. This innovative approach leverages machine learning algorithms and \
natural language processing techniques to identify subtle changes in speech and language that are indicative of \
cognitive decline.")

#q5
qs.append("The primary target users of our AI-based diagnostic tool are healthcare providers, \
including neurologists, geriatricians, and primary care physicians. Additionally, our solution \
could be integrated into telemedicine platforms, enabling remote monitoring and assessment of \
patients at risk for AD. The potential market for our technology includes hospitals, clinics, and \
long-term care facilities, as well as pharmaceutical companies and research institutions involved in \
AD research and drug development.")

# Project narrative
# skip
qs.append("Developing AI-based tool that analyzes speech patterns for early signs of AD.  Should be used by clinicians \
to improve patient outcomes.")

# Personal statement
qs.append("PI")
qs.append("Early-stage investigator")
qs.append("MacArthur genius grant recipient")
qs.append("Trustworthy AI systems, clinical tools")

# Contributions to science
qs.append("Background: Around the world, populations are aging and there is a growing concern about ways that \
older adults can maintain their health and well-being while living in their homes. \
Objectives: The aim of this paper was to conduct a systematic literature review to determine: (1) the \
levels of technology readiness among older adults and, (2) evidence for smart homes and home-based \
health-monitoring technologies that support aging in place for older adults who have complex needs. \
Results: We identified and analyzed 48 of 1863 relevant papers. Our analyses found that: (1) technologyreadiness level for smart homes and home health monitoring technologies is low; (2) the highest level of \
evidence is 1b (i.e., one randomized controlled trial with a PEDro score ≥6); smart homes and home health \
monitoring technologies are used to monitor activities of daily living, cognitive decline and mental health, \
and heart conditions in older adults with complex needs; (3) there is no evidence that smart homes and\
home health monitoring technologies help address disability prediction and health-related quality of life,\
or fall prevention; and (4) there is conflicting evidence that smart homes and home health monitoring\
technologies help address chronic obstructive pulmonary disease.\
Conclusions: The level oftechnology readiness for smart homes and home health monitoring technologies\
is still low. The highest level of evidence found was in a study that supported home health technologies\
for use in monitoring activities of daily living, cognitive decline, mental health, and heart conditions in\
older adults with complex needs.")
qs.append("")
qs.append("")
qs.append("")
          
# this is the information that needs to be sent from the page to the sbir function
page_data = {}
page_data['email'] = "brian3000@gmail.com"
page_data['branch'] = '35zp'
page_data['prompt_header'] = '''
[Applicant Summary]: No information provided
[Agency Summary]:No information provided
[RFP Summary]: No information provided
'''
page_data['model_title'] = "Humor"
page_data['project_id'] = "1683121929445x949068210706579500"
    

## this code will live in google cloud function as the sbir_single_shot
## for each page we look up the prompt, pop off user inputs or cut/paste, send to 
## gpt4, and store the output

In [16]:
DEBUG = True
scratchpad = {}
counter = 0
for page in nih_sbir:
    uid = page.uid
    url = 'https://grantedai.com/version-'+page_data['branch']+'/api/1.1/obj/model/' + uid
    x = requests.get(url)
    model_data = x.json()['response']
    
    # construct body from model_data and project_data
    body = ""
    if model_data['include_header']: body += page_data['prompt_header']
    body += model_data['prompt']
    for i in range(1,page.num_q+1):
        if page.replace_q and i in page.replace_q.keys():
            body = body.replace("q%dinput" % i,scratchpad[page.replace_q[i]])
        else: 
            q = qs.pop(0)
            body = body.replace("q%dinput" % i,q)
                              
    # Data to be sent
    data = {
        "email": page_data['email'],
        "branch": page_data['branch'],
        "debug": "True",
        "body" : body,
        "model_title": page_data['model_title'],
        "project_id": page_data['project_id']
      }

    response = chat_complete_openai(data, my_secret_value)
    text = response['text']
    if page.result not in scratchpad.keys():
        scratchpad[page.result] = ""
    scratchpad[page.result] += text + "\n"

Sending to OpenAI with body 
[Applicant Summary]: No information provided
[Agency Summary]:No information provided
[RFP Summary]


KeyboardInterrupt: 

## this is what all of the output concatenated together look like.  we'll need some additional formatting presumably

In [11]:
import bbcode
from IPython.display import display, HTML

def display_bbcode(bbcode_string):
    parser = bbcode.Parser()
    html_string = parser.format(bbcode_string)
    display(HTML(html_string))

# usage
display_bbcode(scratchpad['Output'])

In [17]:
uid = '1688680831047x583143253860941800'
url = 'https://grantedai.com/version-'+page_data['branch']+'/api/1.1/obj/project/' + uid
x = requests.get(url)
project_data = x.json()['response']
project_data['prompt_header']

"'''\n[Applicant Summary]:\xa0No information provided\n[Agency Summary]:The text provided does not give a detailed description of Google's mission. However, Google is a multinational technology company specializing in Internet-related services and products. These include search engines, online advertising technologies, cloud computing, software, and hardware. The mission of Google is to organize the world's information and make it universally accessible and useful.\n[RFP Summary]:\xa0The text provided does not contain any information about a funding opportunity. It appears to be a list of Google's services and features, such as Google Search, Images, Maps, Play, YouTube, News, Gmail, and Drive, along with some site navigation options.\n'''"

In [ ]:
 https://grantedai.com/version-35zp/api/1.1/obj/project/1688676453367x954995506059935700
2023-07-06 16:48:47.062 EDT
mainffhyyfccoudh 1688676453367x954995506059935700